In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Initiating Serengeti-E250 Model

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [3]:
model = AutoModelForMaskedLM.from_pretrained("UBC-NLP/serengeti-E250")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at UBC-NLP/serengeti-E250 and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForMaskedLM(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(250000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

# Loading Dataset

In [30]:
import pandas as pd

dataset = pd.read_csv('/content/drive/MyDrive/Research/Uni of Pretoria /KINNEWS/kr_train_clean.csv')
dataset['tweet']

0              Hhhhhh ntabyihogoza ubu x abo yishe ban...
1        Amahano Ni impanuka inkangu inzara Muyite izi...
2       Ese umuntu aguhaye miliyoni 7 zidorali ngo ary...
3       Ugira amagambo kandi Ubwo wasanga nawe byaguta...
4       Ukuntu inama zose zikomeye zirikubera Mu Rwand...
                              ...                        
3297    Tugukunda kurusha mukobwa mwiza Amahoro ibyish...
3298    Sobanukirwa IBYIZA MASSAGE IFITEYE UMUBIRI  ht...
3299    Mushobora kugira uruhare muri iki kiganiro mut...
3300    22 Ntuduhane mu bitwoshya Ahubwo udukize Umubi...
3301          Ni umuyobozi wintangarugero aho ageze ho...
Name: tweet, Length: 3302, dtype: object

# E250 Tokenizer vs Custom Electra Tokenizer

In [36]:
import time

serengeti_model = AutoModelForMaskedLM.from_pretrained("UBC-NLP/serengeti-E250")
serengeti_tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/serengeti-E250")

custom_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Research/Tokenizers/electra-tokenizer")

text = "Ese umuntu aguhaye miliyoni 7 zidorali ngo aryamane numugore wawe cg umukunzi wawe wabyemera"

# Measure tokenization speed
start_time = time.time()
_ = serengeti_tokenizer.tokenize(text)
serengeti_time = time.time() - start_time

start_time = time.time()
_ = custom_tokenizer.tokenize(text)
custom_time = time.time() - start_time

print(f"Serengeti Tokenization Time: {serengeti_time} seconds")
print(f"Custom Tokenizer Time: {custom_time} seconds")

# Evaluate tokenization accuracy (compare tokenized outputs)

# Example tokenization accuracy test
serengeti_tokens = serengeti_tokenizer.tokenize(text)
custom_tokens = custom_tokenizer.tokenize(text)

print("Serengeti Tokens:", serengeti_tokens)
print("Custom Tokens:", custom_tokens)

# Evaluate vocabulary coverage and special token handling
serengeti_vocab_size = len(serengeti_tokenizer)
custom_vocab_size = len(custom_tokenizer)

print(f"Serengeti Vocabulary Size: {serengeti_vocab_size}")
print(f"Custom Vocabulary Size: {custom_vocab_size}")

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at UBC-NLP/serengeti-E250 and are newly initialized: ['generator_lm_head.bias', 'generator_predictions.LayerNorm.bias', 'generator_predictions.LayerNorm.weight', 'generator_predictions.dense.bias', 'generator_predictions.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Serengeti Tokenization Time: 0.0008454322814941406 seconds
Custom Tokenizer Time: 0.0007367134094238281 seconds
Serengeti Tokens: ['ese', 'umuntu', 'agu', '##haye', 'miliyoni', '7', 'zid', '##oral', '##i', 'ngo', 'arya', '##mane', 'numu', '##gore', 'wawe', 'cg', 'umukunzi', 'wawe', 'wabye', '##mera']
Custom Tokens: ['▁Ese', '▁umuntu', '▁aguhaye', '▁miliyoni', '▁7', '▁zi', 'dorali', '▁ngo', '▁aryamane', '▁numugore', '▁wawe', '▁cg', '▁umukunzi', '▁wawe', '▁wabyemera']
Serengeti Vocabulary Size: 250000
Custom Vocabulary Size: 162272


# Masked LM Predictions

In [38]:
fill_mask_e250 = pipeline('fill-mask', model=model, tokenizer = serengeti_tokenizer)
fill_mask_custom = pipeline('fill-mask', model=model, tokenizer = custom_tokenizer)

In [40]:
dataset['tweet'][10]

'Mf iziko yasibye EP ya  kuri YouTube nizibeshye go ikadagire muri kkblock seriously Rwanda'

In [54]:
text = dataset['tweet'][10]
text

'Mf iziko yasibye EP ya  kuri YouTube nizibeshye go ikadagire muri kkblock seriously Rwanda'

In [56]:
masked_text = 'Mf iziko yasibye EP ya  kuri YouTube [MASK] go ikadagire muri kkblock seriously Rwanda'

In [57]:
fill_mask_custom(masked_text)

[{'score': 0.00021190218103583902,
  'token': 1452,
  'token_str': 'cha',
  'sequence': 'Mf iziko yasibye EP ya  kuri YouTube  cha go ikadagire muri kkblock seriously Rwanda'},
 {'score': 0.00020089434110559523,
  'token': 99394,
  'token_str': 'twapfa',
  'sequence': 'Mf iziko yasibye EP ya  kuri YouTube  twapfa go ikadagire muri kkblock seriously Rwanda'},
 {'score': 0.00018856461974792182,
  'token': 112802,
  'token_str': 'bwongerereza',
  'sequence': 'Mf iziko yasibye EP ya  kuri YouTube  bwongerereza go ikadagire muri kkblock seriously Rwanda'},
 {'score': 0.00017239178123418242,
  'token': 120025,
  'token_str': 'gutunguranye',
  'sequence': 'Mf iziko yasibye EP ya  kuri YouTube  gutunguranye go ikadagire muri kkblock seriously Rwanda'},
 {'score': 0.00016051602142397314,
  'token': 187635,
  'token_str': '',
  'sequence': 'Mf iziko yasibye EP ya  kuri YouTube  go ikadagire muri kkblock seriously Rwanda'}]

In [58]:
fill_mask_e250(masked_text)

[{'score': 0.00035297643626108766,
  'token': 122034,
  'token_str': '##chisha',
  'sequence': 'mf iziko yasibye ep ya kuri youtubechisha go ikadagire muri kkblock seriously rwanda'},
 {'score': 0.00033142438041977584,
  'token': 197903,
  'token_str': '##chishwa',
  'sequence': 'mf iziko yasibye ep ya kuri youtubechishwa go ikadagire muri kkblock seriously rwanda'},
 {'score': 0.00028024750645272434,
  'token': 71061,
  'token_str': 'bans',
  'sequence': 'mf iziko yasibye ep ya kuri youtube bans go ikadagire muri kkblock seriously rwanda'},
 {'score': 0.0002296040183864534,
  'token': 4390,
  'token_str': 'makala',
  'sequence': 'mf iziko yasibye ep ya kuri youtube makala go ikadagire muri kkblock seriously rwanda'},
 {'score': 0.000224229326704517,
  'token': 232570,
  'token_str': 'hg',
  'sequence': 'mf iziko yasibye ep ya kuri youtube hg go ikadagire muri kkblock seriously rwanda'}]

# Best Predictions vs Worst Predictions


In [63]:
sorted_predictions = sorted(fill_mask_e250(masked_text), key=lambda x: x['score'])

# Print best (highest score) and worst (lowest score) predictions
print("Best prediction:")
print(f"Token: {sorted_predictions[-1]['token_str']}, Score: {sorted_predictions[-1]['score']}")
print(f"Sequence: {sorted_predictions[-1]['sequence']}\n")

print("Worst prediction:")
print(f"Token: {sorted_predictions[0]['token_str']}, Score: {sorted_predictions[0]['score']}")
print(f"Sequence: {sorted_predictions[0]['sequence']}")

Best prediction:
Token: ##chisha, Score: 0.00035297643626108766
Sequence: mf iziko yasibye ep ya kuri youtubechisha go ikadagire muri kkblock seriously rwanda

Worst prediction:
Token: hg, Score: 0.000224229326704517
Sequence: mf iziko yasibye ep ya kuri youtube hg go ikadagire muri kkblock seriously rwanda


In [64]:
sorted_predictions = sorted(fill_mask_custom(masked_text), key=lambda x: x['score'])

# Print best (highest score) and worst (lowest score) predictions
print("Best prediction:")
print(f"Token: {sorted_predictions[-1]['token_str']}, Score: {sorted_predictions[-1]['score']}")
print(f"Sequence: {sorted_predictions[-1]['sequence']}\n")

print("Worst prediction:")
print(f"Token: {sorted_predictions[0]['token_str']}, Score: {sorted_predictions[0]['score']}")
print(f"Sequence: {sorted_predictions[0]['sequence']}")

Best prediction:
Token: cha, Score: 0.00021190218103583902
Sequence: Mf iziko yasibye EP ya  kuri YouTube  cha go ikadagire muri kkblock seriously Rwanda

Worst prediction:
Token: , Score: 0.00016051602142397314
Sequence: Mf iziko yasibye EP ya  kuri YouTube  go ikadagire muri kkblock seriously Rwanda
